In [1]:
import torch
from torch_geometric.data import Data
from torch_geometric.datasets import EllipticBitcoinDataset

from ogb.nodeproppred import PygNodePropPredDataset

In [6]:
from pathlib import Path
from torch_geometric.data import Dataset
import networkx as nx
import numpy as np
import torch_geometric

def built_dblp():
    path = Path('../data/dblp-hard-citation-graph/dblp-hard/')
    y = np.load(path/"y.npy")
    x = np.load(path/"X.npy")
    node_year = np.load(path/"t.npy")
    nx_graph = nx.read_adjlist(path/"adjlist.txt", nodetype=int)
    data = torch_geometric.utils.from_networkx(nx_graph)
    data.x = torch.tensor(x)
    data.y = torch.unsqueeze(torch.tensor(y), 1)
    data.node_year = torch.unsqueeze(torch.tensor(node_year),1)
    num_classes = np.unique(y).shape[0]
    #ds = Dataset(data, num_classes)
    print(num_classes)
    return data

In [2]:
elliptic = EllipticBitcoinDataset(root='./data/Elliptic/')
ogbn_arxiv = PygNodePropPredDataset(name = "ogbn-arxiv", root = './data/Ogbn/')
dblp = built_dblp()

In [4]:
from torch_geometric.utils import homophily
edge_index = elliptic.edge_index
y = elliptic.y
homophily(edge_index, y, method='edge')

0.7113396525382996